## Local MNIST Training with MXNet and Gluon

### Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local`.  You could also use `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance, but then you'll need to set `train_instance_count=1` since distributed, local, GPU training is not yet supported.

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).  Running the setup.sh script below will handle this for you.

**Note, you can only run a single local notebook at one time.**

In [ ]:
import sys
! {sys.prefix}/bin/pip install mxnet

In [ ]:
!/bin/bash /home/ec2-user/SageMaker/AWS-ML-Certification/__my_study/sagemaker/gluon/mxnet_gluon_mnist/setup.sh


In [ ]:
! echo $PWD

### Overview

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker local mode using MXNet and the Gluon API.

In [7]:
import os
import subprocess
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

instance_type = 'local'

# if subprocess.call('nvidia-smi') == 0:
#     ## Set type to GPU if one is present
#     instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

# role = get_execution_role()
role = 'arn:aws:iam::868024899531:role/service-role/AmazonSageMaker-ExecutionRole-20200530T112594'

Instance type = local


## Download training and test data

In [8]:
gluon.data.vision.MNIST('./data/train', train=True)
gluon.data.vision.MNIST('./data/test', train=False)

## Uploading the data

We use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value `inputs` identifies the location -- we will use this later when we start the training job.

In [9]:
bucket = 'md-labs-bucket'
prefix = 'md-labs-mxnet-gluon/data/mnist'

inputs = sagemaker_session.upload_data(bucket=bucket, path='data', key_prefix=prefix)
print(inputs)


s3://md-labs-bucket/md-labs-mxnet-gluon/data/mnist


## Implement the training function

We need to provide a training script that can run on the SageMaker platform. The training scripts are essentially the same as one you would write for local training, except that you need to provide a `train` function. The `train` function will check for the validation accuracy at the end of every epoch and checkpoints the best model so far, along with the optimizer state, in the folder `/opt/ml/checkpoints` if the folder path exists, else it will skip the checkpointing. When SageMaker calls your function, it will pass in arguments that describe the training environment. Check the script below to see how this works.

The script here is an adaptation of the [Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) provided by the [Apache MXNet](https://mxnet.incubator.apache.org/) project. 

In [12]:
# !cat 'mnist.py'
!pygmentize '/home/ec2-user/SageMaker/AWS-ML-Certification/__my_study/sagemaker/gluon/mxnet_gluon_mnist/mnist.py'

from __future__ import print_function

import argparse
import logging
import os

import mxnet as mx
from mxnet import gluon, autograd
from mxnet.gluon import nn
import numpy as np
import json
import time


logging.basicConfig(level=logging.DEBUG)

# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #


def train(args):
    # SageMaker passes num_cpus, num_gpus and other args we can use to tailor training to
    # the current container environment, but here we just use simple cpu context.
    ctx = mx.cpu()

    # retrieve the hyperparameters we set in notebook (with some defaults)
    batch_size = args.batch_size
    epochs = args.epochs
    learning_rate = args.learning_rate
    momentum = args.momentum
    log_interval = args.log_interval

    num_gpus = int(os.environ['SM_NUM_GPUS'])
    current_host = args.current_host
    hosts = args.hosts
 

## Run the training script on SageMaker

The ```MXNet``` class allows us to run our training function on SageMaker local mode. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type.  This is the the only difference from [mnist_with_gluon.ipynb](./mnist_with_gluon.ipynb).  Instead of ``train_instance_type='ml.c4.xlarge'``, we set it to ``train_instance_type='local'``.  For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

In [11]:
m = MXNet("/home/ec2-user/SageMaker/AWS-ML-Certification/__my_study/sagemaker/gluon/mxnet_gluon_mnist/mnist.py",
          role=role,
          train_instance_count=1,
          train_instance_type=instance_type,
          framework_version="1.6.0",
          py_version="py3",
          hyperparameters={'batch-size': 100,
                           'epochs': 20,
                           'learning-rate': 0.1,
                           'momentum': 0.9,
                           'log-interval': 100})

After we've constructed our `MXNet` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.

In [13]:
m.fit(inputs)

Using the short-lived AWS credentials found in session. They might expire while running.
Command output: 


Exception: Failed to run docker,images,-q,763104351884.dkr.ecr.us-east-2.amazonaws.com/mxnet-training:1.6.0-cpu-py3

After training, we use the MXNet object to deploy an MXNetPredictor object. This creates a SageMaker endpoint locally that we can use to perform inference. 

This allows us to perform inference on json encoded multi-dimensional arrays. 

In [ ]:
predictor = m.deploy(initial_instance_count=1, instance_type=instance_type)

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a 'data' variable in this notebook, which we can then pass to the mxnet predictor.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

The predictor runs inference on our input data and returns the predicted digit (as a float value, so we convert to int for display).

In [ ]:
response = predictor.predict(data)
print(int(response))

## Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
predictor.delete_endpoint()